In [ ]:
%load_ext autoreload
%autoreload 2

In [28]:
import main_constants as ct
import json
from services.index import Index
import unidecode
from tqdm import tqdm
import torch
from services import sql
import sqlite3
import json
import pickle
import pandas as pd
import spacy

In [ ]:
# check if embeddings are right.
from retrieval.neural.modules.encoders import MaxPoolEncoder, MeanPoolEncoder
from retrieval.neural.modules.scorers import LinearLogisticRegression
encoder = MaxPoolEncoder(50)
encoder.embed(torch.tensor(1).view(1,1))

In [ ]:
# print term frquencies to see if they are ordered.
with open('./data/index/token2id.tar', 'rb') as file:
    t2i = pickle.load(file)
with open('./data/index/id2tf.tar', 'rb') as file:
    i2tf = pickle.load(file)
for t in list(t2i.keys())[0:100]:
    print(f'{t}\t\t{t2i[t]}\t{i2tf[t2i[t]]}')

In [ ]:
# time how fast pyndri can query: very slow...
index = Index('tfidf')
# as opposed to db lookup.
def fn():
    with sqlite3.connect('./data/term_retrievals/uni_tfidf.dummy/retrievals.sqlite') as db:
        (q_id, target_titles, doc_int_ids) = db.cursor() \
            .execute(sql.get_question_by_qid(), ('5a7a52745542996c55b2dd4f',)) \
            .fetchone()
        return pickle.loads(doc_int_ids)[:1000]
%timeit index.unigram_query('What is anarchism?', 1000)
%timeit fn()

In [ ]:
# check run looks good.
run = Run()
run.add_ranking("a", {"a1": 1})
run.add_ranking("b", {"b1": 1, "b2": 2})
run.update_ranking("d", "d1", 3)
run.update_rankings({"c": {"c1": 3}, "b": {"b3": 3}})
run

In [ ]:
df = pd.read_pickle('./data/candidates/tfidf.dev.dummy.gzip', compression='gzip')
df.head()

In [8]:
ent_model = spacy.load('xx_ent_wiki_sm')

In [9]:
index = Index()

INFO:root:[2018-12-10 02:14:27.037000]	[28824]	Loading index ./data/index/indri with default query environment.
INFO:root:[2018-12-10 02:14:31.886280]	[28824]	Not loading stopwords.
INFO:root:[2018-12-10 02:14:40.501740]	[28824]	Loaded index in 0:00:13.463938.


In [31]:
db = sqlite3.connect(ct.DOCUMENT_DB)
cursor = db.cursor()
(doc_1,) = cursor.execute("SELECT text FROM documents WHERE id = ?", (index.title2wid["Scott Derrickson"],)).fetchone()
(doc_2,) = cursor.execute("SELECT text FROM documents WHERE id = ?", (index.title2wid["Ed Wood"],)).fetchone()
question = "Were Scott Derrickson and Ed Wood of the same nationality?"

In [33]:
question_parse = ent_model(question)
doc_1_parse = ent_model(doc_1)
doc_2_parse = ent_model(doc_2)

In [34]:
spacy.displacy.render([question_parse, doc_1_parse, doc_2_parse], style='ent', jupyter=True)